<h1><center>Build <a href="https://howsmynhs.co.uk/">HowsMyNHS</a> website</center></h1>

This is the main script used to automatically generate the html for <a href="https://howsmynhs.co.uk/">HowsMyNHS</a>.

In [1]:
import build_website as bw
waiting_data = 'NHSwaiting.npy'
bed_data = 'NHSbeds.npy'

<h2>Generate Plots</h2>

In [2]:
bw.plotWaitingData(waiting_data)
#bw.plotBedData(bed_data)

Generating 4 hour waiting time graphs... Merged Trust!!
104 104
[False False False False  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]
[2020.25       2020.16666667 2020.08333333 2020.         2019.66666667
 2019.58333333 2019.5        2019.41666667 2019.33333333 2019.25
 2019.08333333 2019.         2018.91666667 2018.83333333 2018.75
 2018.66666667 2018.58333333 2018.5        2018.41666667 2018.33333333
 2018.25       2018.16666667 2

<h2>Make homepage</h2>

In [2]:
bw.MakeHomepage(waiting_data, bed_data)

Building homepage... Bedfordshire Hospitals NHS Foundation Trust
Done


<h2>Make all the other pages</h2>

In [3]:
bw.build_trust_pages(waiting_data, bed_data)

Building trust pages... Done.


In [3]:
bw.get_names(waiting_data)

array(['England', nan, 'Bedfordshire Hospitals NHS Foundation Trust',
       'Cambridge University Hospitals NHS Foundation Trust',
       'Clacton Hospital', 'East And North Hertfordshire NHS Trust',
       'East Suffolk And North Essex NHS Foundation Trust',
       'Fryatt Hospital', 'Hertfordshire Community NHS Trust',
       'Herts Urgent Care (Ascots Lane)',
       'James Paget University Hospitals NHS Foundation Trust',
       'Mid And South Essex NHS Foundation Trust',
       'Milton Keynes University Hospital NHS Foundation Trust',
       'Norfolk And Norwich University Hospitals NHS Foundation Trust',
       'North West Anglia NHS Foundation Trust',
       'Putnoe Medical Centre Walk In Centre',
       'The Princess Alexandra Hospital NHS Trust',
       "The Queen Elizabeth Hospital, King's Lynn, NHS Foundation Trust",
       'West Hertfordshire Hospitals NHS Trust',
       'West Suffolk NHS Foundation Trust',
       'Barking, Havering And Redbridge University Hospitals NHS Tr